For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

1) Data cleaning / processing / language parsing

2) Create features using two different NLP methods: For example, BoW vs tf-idf.

3) Use the features to fit supervised learning models for each feature set to predict the category outcomes.

4) Assess your models using cross-validation and determine whether one model performed better.

5) Pick one of the models and try to increase accuracy by at least 5 percentage points.

Write up your report in a Jupyter notebook. Be sure to explicitly justify the choices you make throughout, and submit it below.

# Data uplaoding/cleaning

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/kyleknoebel/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!



    Linking successful
    /anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Load and clean the data.
bryant = gutenberg.raw('bryant-stories.txt')
emma = gutenberg.raw('austen-emma.txt')
# The Chapter indicator is idiosyncratic
bryant = re.sub(r'Chapter \d+', '', bryant)
emma = re.sub(r'Chapter \d+', '', emma)
# clean text   
bryant = text_cleaner(bryant)
emma = text_cleaner(emma)
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
bryant_doc = nlp(bryant)
emma_doc = nlp(emma)
# Group into sentences.
bryant_sents = [[sent, 'Bryant'] for sent in bryant_doc.sents]
emma_sents = [[sent, 'Austen'] for sent in emma_doc.sents]
# fix errors of visualization
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(bryant_sents + emma_sents)
sentences.head()

,0,1
0,"(TWO, LITTLE, RIDDLES, IN, RHYME)",Bryant
1,"(There, 's, a, garden, that, I, ken, ,, Full, ...",Bryant
2,"((, Flax, ., ))",Bryant
3,"(From, house, to, house, he, goes, ,, A, messe...",Bryant
4,"((, The, path, ., ))",Bryant


# Feature engineering

## BOW

In [4]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
emmawords = bag_of_words(emma_doc)
bryantwords = bag_of_words(bryant_doc)

# Combine bags to create a set of unique words.
common_words = set(emmawords + bryantwords)

In [5]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000
Processing row 7500
Processing row 8000
Processing row 8500
Processing row 9000
Processing row 9500
Processing row 10000
Processing row 10500
Processing row 11000
Processing row 11500


,huge,agin,dead,tower,detain,force,material,millstone,speech,mistake,...,certainly,fox,mountain,ireland,spring,bubble,lose,cornfield,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(TWO, LITTLE, RIDDLES, IN, RHYME)",Bryant
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, 's, a, garden, that, I, ken, ,, Full, ...",Bryant
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, Flax, ., ))",Bryant
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(From, house, to, house, he, goes, ,, A, messe...",Bryant
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, The, path, ., ))",Bryant


# Prediction with machine learning

## tfidf

In [6]:
import nltk
import re
from sklearn.model_selection import train_test_split

#reading in the data, this time in the form of paragraphs
bryant2=gutenberg.paras('bryant-stories.txt')
#processing
bryant_paras=[]
for paragraph in bryant2:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    bryant_paras.append(' '.join(para))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(bryant_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
bryant_paras_tfidf=vectorizer.fit_transform(bryant_paras)
print("Number of features: %d" % bryant_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(bryant_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space from 1379 to 130.
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)

Number of features: 1116
Percent variance captured by all components: 57.16653568777341


## BOW

In [7]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

# Import deep learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate

# Establish and fit the model, with ...
mlp = MLPClassifier(hidden_layer_sizes=(60, 60, 60), activation='identity', learning_rate='constant')
mlp.fit(X_train, y_train)
cv_results = cross_validate(mlp, X, Y, cv=5, return_train_score=False)
print('Training set score:', mlp.score(X_train, y_train))
print('\nTest set score:', mlp.score(X_test, y_test))
print('\nCross validation scores:', cv_results)

Training set score: 0.9862059158134243

Test set score: 0.9102153977393901

Cross validation scores: {'fit_time': array([35.29248691, 45.63367987, 51.26294398, 43.16702485, 38.21238899]), 'score_time': array([0.20670104, 0.17262697, 0.26528311, 0.1624043 , 0.24884725]), 'test_score': array([0.87420043, 0.86823028, 0.87926621, 0.86006826, 0.86043534])}


# Improve accuracy by 5% - tfidf

In [8]:
X_train, X_test = train_test_split(bryant_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.75, # drop words that occur in more than half the paragraphs
                             min_df=4, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
bryant_paras_tfidf=vectorizer.fit_transform(bryant_paras)
print("Number of features: %d" % bryant_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(bryant_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space from 1379 to 130.
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)

Number of features: 498
Percent variance captured by all components: 69.26323998522915
